# Uber Creative Exercise - Clarissa Simoyama David

## Tarefa atribuída

O dado que recebi como tarefa fora um arquivo de extensão PDF com meu própiro currículo. Estendendo este problema para o mundo da Uber, imaginei como deve ser feito a seleção de candidatos à motoristas para a companhia. Logo pude ver com clareza um problema que poderia ser solucionado: novos candidatos que estariam interessados a trabalharem juntos à companhia enviam seus currículos e, de alguma forma, é necessário a automatização da extração das informações do que eles enviam à empresa. Foi pensando neste problema que desenvolvi a solução que será explicada nas subseções seguintes.

## Solução

Para solucionar o problema da extração de informações de currículos em PDF, pensei em uma solução automatizada utilizando Python. 

A primeira etapa consiste em transformar o arquivo PDF em um arquivo TXT, para se ter uma forma mais fácil de manipulação. O script pdf2txt.py pertence à biblioteca pdfminer.

In [57]:
%%bash
pdf2txt.py Profile_\(16\).pdf > profile_16.txt

Após a transformação, é feita a leitura deste arquivo para ser manipulado.

In [2]:
file_txt_to_read = open("profile_16.txt", "r")
profile_in_txt = file_txt_to_read.read()

O resultado obtido da transformação do arquivo PDF para o TXT:

In [3]:
profile_in_txt

'Contatar\n\nwww.linkedin.com/in/clarissa-\ndavid-a10824170 (LinkedIn)\ngithub.com/ClaDavid (Other)\n\nPrincipais competências\nAprendizado de máquina\nMineração de dados\nProgramação de computadores\n\nLanguages\nEspanhol (Limited Working)\nInglês (Full Professional)\nPortuguês brasileiro (Native or\nBilingual)\n\nCertifications\nSupercomputer with NVIDIA K20\nboards\nIntroduction to Object-Oriented\nProgramming\n\nHonors-Awards\nACM International Collegiate\nProgramming Contest/XXI Maratona\nde Programação\nInvestigation of the use of text\nmining to discover the meaning of\nclusters in biological entities\n\n\xa0\n\n\xa0\n\n\xa0\n\nClarissa David\n\nData Scientist\nSão Bernardo do Campo, São Paulo, Brasil\n\nResumo\nGraduada em Bacharelado em Ciência e Tecnologia e Bacharelado\nem Ciência da Computação, ambos pela Universidade Federal\ndo ABC (UFABC), a atividade que mais possuo afinidade é\nobter diversos tipos de conhecimentos novos e colocá-los em\nprática. Amo a profissão que es

Com o arquivo TXT carregado, vem a parte de criação das funções para a extração automática das informações. Para isto foi criado uma classe, para uma melhor organização e encapsulamento. As duas primeiras funções, remove_strange e no_break_line são funções de pré-processamento. Ineretemente a maior parte dos dados vem com "lixos" e caracters que não queremos e podemos descartar ou substituir, sendo necessário um pré-processamento dos dados. 

As funções seguintes são responsáveis pelas extrações das informações: getContatos é responsável por extrair as informações de contatos, getExperiencia é responsável por extrair informações de experiências profissionais anteriores, getFormacaoAcademica é responsável por extrair informações de formações acadêmicas, getIdiomas é responsável por extrair quais idiomas a pessoa domina e getResumo é responsável por extrair o resumo dado no currículo.

In [78]:
import numpy as np
import pandas as pd
import re
import unicodedata

class ExtracaoInformacaoCV(object):
    
    def remove_strange(self, sentence):
        sentence_sub = sentence.replace(u'\x0c', u' ')
        sentence_sub = sentence_sub.replace("Page 2 of 2", "")
        sentence_sub = sentence_sub.replace("Page 1 of 2", "")
        sentence_sub = sentence_sub.replace("·", " ")
        sentence_sub = sentence_sub.replace(" - ", "-")
        sentence_sub = sentence_sub.replace(" -", "-")
        sentence_sub = sentence_sub.replace("- ", "-")
        sentence_sub = unicodedata.normalize("NFKD", sentence_sub)
        return sentence_sub

    def no_break_line(self, file_to_be_clean):
        split_break_line = re.compile('\n \n').split(file_to_be_clean)
        clean_break_line = [re.sub(r'\n', ' ', bl) for bl in split_break_line]
    
        return(clean_break_line)
    
    def getContatos(self, inputString):
        cleaning_data = ''.join(no_break_line(inputString))
        data = remove_strange(cleaning_data)
        data_string = ''.join(no_break_line(data))
        regex_contatos = r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"
        lista_contato = []
        for match in re.finditer(regex_contatos, data_string):
            lista_contato.append(match.group(0))
        if lista_contato != []:
            return lista_contato
        else:
            return "Contato não informado"
    
    def getExperiencia(self, inputString):
        data = remove_strange(inputString)
        data = data.replace('\n', 'ENTER')
        match = re.search(r"Experiência(.*?)\.ENTERENTER", data, re.IGNORECASE)
        if match != None:
            experiencia = match.group(1)
        return experiencia.replace('ENTER', ' ').lstrip()
    
    def getFormacaoAcademica(self, inputString):
        data = remove_strange(inputString)
        data = data.replace('\n', 'ENTER')
        match = re.search(r"Formação\s*acadêmica(.*?)$", data, re.IGNORECASE)
        if match != None:
            formacao_academica = match.group(1)
        return formacao_academica.replace('ENTER', ' ').strip()
    
    def getIdiomas(self, inputString):
        data = remove_strange(inputString)
        data = data.replace('\n', 'ENTER')
        match = re.search(r"Languages(.*?)ENTERENTER", data, re.IGNORECASE)
        if match != None:
            idiomas = match.group(1)
        return idiomas.replace('ENTER', ' ').lstrip()
    
    def getResumo(self, inputString):
        data = remove_strange(profile_in_txt)
        data = data.replace('\n', 'ENTER')
        match = re.search(r"Resumo(.*?)ENTERENTER", data, re.IGNORECASE)
        if match != None:
            resumo = match.group(1)
        return resumo.replace('ENTER', ' ').strip()

Com as funções feitas e a classe pronta, um objeto dela é instanciado e um dataframe é feito para poder guardar essas informações que serão extraídas, podendo posteriormente ser salvo em um banco de dados e consultado para aplicações futuras.

In [89]:
objeto_extracao = ExtracaoInformacaoCV()

colunas = ['contatos', 'experiencia', 'formacao_academica', 'idiomas', 'resumo']
df_resultado_extracao = pd.DataFrame(columns=colunas)
lista_extracoes_resultantes = []
lista_extracoes_resultantes.append(objeto_extracao.getContatos(profile_in_txt))
lista_extracoes_resultantes.append(objeto_extracao.getExperiencia(profile_in_txt))
lista_extracoes_resultantes.append(objeto_extracao.getFormacaoAcademica(profile_in_txt))
lista_extracoes_resultantes.append(objeto_extracao.getIdiomas(profile_in_txt))
lista_extracoes_resultantes.append(objeto_extracao.getResumo(profile_in_txt))
df_resultado_extracao.loc[len(df_resultado_extracao), :] = lista_extracoes_resultantes

## Resultado

A seguir temos o resultado da técnica de extração de informações do currículo. Com as funções de extrações obtêm-se um dataframe de resultado contendo os contatos, experiências profissionais, formação acadêmica, idiomas e o resumo, tudo que estava contido no currículo, mas de forma estruturada automaticamente. As técnicas utilizadas para extrações foram técnicas de expressões regulares e processamento de linguagem natural.

In [88]:
pd.set_option('display.max_colwidth', -1)
df_resultado_extracao

,contatos,experiencia,formacao_academica,idiomas,resumo
0,"[www.linkedin.com/in/clarissa-david-a10824170, github.com/ClaDavid]",Semantix Data Scientist fevereiro de 2019 - Present Thales Desenvolvedora Jr outubro de 2018 - janeiro de 2019 (4 meses) Freelance Consultora agosto de 2018 - agosto de 2018 (1 mês) Consultoria para análise estatística de dados psicológicos,"Universidade Federal do ABC-UFABC Mestrado, Ciência da Computação (2017 - 2019) Universidade Federal do ABC-UFABC Bacharelado, Ciência da Computação (2012 - 2018) Universidade Federal do ABC-UFABC Bacharelado, Ciência e Tecnologia (2012 - 2016)",Espanhol (Limited Working) Inglês (Full Professional) Português brasileiro (Native or Bilingual),"Graduada em Bacharelado em Ciência e Tecnologia e Bacharelado em Ciência da Computação, ambos pela Universidade Federal do ABC (UFABC), a atividade que mais possuo afinidade é obter diversos tipos de conhecimentos novos e colocá-los em prática. Amo a profissão que estudei e gosto de aprender novas linguagens e testá-las, sendo a programação um dos meus hobbies favoritos. Estou em meu segundo ano de mestrado em Ciência da Computação (término 07/2019), pela UFABC, e minha área concentra-se em Machine Learning, Data Mining e NLP."


## Impact in the business

As part of the dynamic of the homework, this subsection will be done in English. Automating the process of information extraction can not only optimize time, but also optimize recourse in the question of people, money and organization. Doing this manually or using natural language process can be susceptible to errors that can cost time and rework.

With this feature, it will be easier to hire people that are more aligned with the company's goals, also making it easier to view information about who works for Uber, generating greater control and comfort for both drivers and passengers.

In the future, this feature can also be a start to something bigger: people and talent analytics, in partnership with the databases that HR has, not only automating the extraction of information, but also improve the quality of all employees and users as a whole.

## Códigos

Os códigos também podem ser encontrados no github, no seguinte repositório: https://github.com/ClaDavid/UberCreativeExercise